#Implementation of A* algorithm on a maze


#### Name: Saladi Naga Sai Srinija
#### Roll no.: 411974

### Create a priority queue for maintaining the frontier required to implement A* on the graph

In [1]:
#Create a class to implement priority queue methods
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]


### Define functions to draw a maze to represent nodes and paths 

In [2]:
# Representation of symbols in each tile of the grid representing the maze
def draw_tile(graph, id, style, width):
    r = "."
    if 'number' in style and id in style['number']: r = "%d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = "<"
        if x2 == x1 - 1: r = ">"
        if y2 == y1 + 1: r = "^"
        if y2 == y1 - 1: r = "v"
      #\033[1m and \033[0m are added to make the letters bold
    if 'start' in style and id == style['start']: r = "\033[1mS  \033[0m"
    if 'goal' in style and id == style['goal']: r = "\033[1mG  \033[0m"
    if 'path' in style and id in style['path']: r = "$"
    if id in graph.walls: r = "w" * width
    return r



In [3]:
# Draw a maze with the given parameters
def draw_maze(graph, width=3, **style):
    for y in range(graph.height):
        for x in range(graph.width):
            print("%%-%ds" % width % draw_tile(graph, (x, y), style, width), end="")
        print()

In [4]:
# Create a square grid to represent maze

class SquareGrid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.walls = []
    
    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id):
        return id not in self.walls
    
    def neighbours(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        if (x + y) % 2 == 0: results.reverse()
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return results



In [5]:
# define the weight of each tile in the grid

class GridWithWeights(SquareGrid):
    def __init__(self, width, height):
        super().__init__(width, height)
        self.weights = {}
    
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)

maze = GridWithWeights(12, 12)
maze.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8),(4,2),(4,3),(4,4),(5,3),(5,4),(6,3),(6,4),(8,6),(8,7),(8,8),(8,9)]
maze.weights = {loc: 3 for loc in [(3, 4), (4, 2),(4, 3), (4, 4), (4, 5), (4, 6),(5, 2),(6, 7),(7, 3), (7, 4), (7, 5),(8,5),(8,9),(9,10),(9,11),(10,8)]}



### Heuristic function to implement A* algorithm using f(n) = g(n) + h(n)

In [6]:
# Heuristic function for the A* search algorithm
# This function gives the manhattan distance between two points
def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

### Implementation code for A* algorithm

In [7]:
# Implementation of A* algorithm for a graph
# Heuristic function taken is the manhattan distance between a given point to the goal
def a_star(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    nodes = {}
    costs = {}
    nodes[start] = None
    costs[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbours(current):
            new_cost = costs[current] + graph.cost(current, next)
            if next not in costs or new_cost < costs[next]:
                costs[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put(next, priority)
                nodes[next] = current
    
    return nodes, costs

### Construct Shortest path from Start node to Goal node




In [8]:
# Reconstruction of path to represent the shortest path in the graph
def reconstruct_path(nodes, start, goal):
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = nodes[current]
    path.append(start)
    path.reverse()
    return path


### Diagramatic representation of A* implementation on a maze

In [9]:
# Defining start and goal points in the maze
start, goal = (1, 1), (10,8)
nodes, costs = a_star(maze, start, goal)


In [10]:
# Representation of paths in the maze
# Walls are represented by w
# path directions are represented by >, <, ^, v towards pointed side of the arrows
draw_maze(maze, width=3, point_to=nodes, start=start, goal=goal)
print()

^  ^  ^  ^  ^  ^  ^  ^  ^  ^  ^  >  
<  S  >  >  >  >  >  >  >  >  >  >  
<  v  >  >  wwwv  v  >  >  >  >  >  
<  v  >  >  wwwwwwwwwv  v  >  >  >  
<  v  >  >  wwwwwwwww<  v  >  >  >  
<  v  >  >  >  >  >  >  v  v  >  >  
<  v  >  >  >  >  >  >  wwwv  >  >  
v  wwwwwwwwwv  >  >  v  wwwv  >  >  
v  wwwwwwwwwv  >  >  >  wwwv  G  .  
v  .  .  .  v  v  v  v  wwwv  >  .  
.  .  .  .  .  .  .  .  .  v  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  



In [11]:

# Representing the costs to each point in the maze
draw_maze(maze, width=3, number=costs, start=start, goal=goal)
print()


2  1  2  3  4  5  6  7  8  9  10 11 
1  S  1  2  3  4  5  6  7  8  9  10 
2  1  2  3  www7  6  7  8  9  10 11 
3  2  3  4  wwwwwwwww10 9  10 11 12 
4  3  4  7  wwwwwwwww13 10 11 12 13 
5  4  5  6  9  10 11 14 13 12 13 14 
6  5  6  7  10 11 12 13 www13 14 15 
7  wwwwwwwww11 12 15 14 www14 15 16 
8  wwwwwwwww12 13 14 15 www15 G  .  
9  .  .  .  13 14 15 16 www16 17 .  
.  .  .  .  .  .  .  .  .  19 .  .  
.  .  .  .  .  .  .  .  .  .  .  .  



In [12]:
# The shortest path found from S to G through A* algorithm
# Path represented by $
draw_maze(maze, width=3, path=reconstruct_path(nodes, start=start, goal=goal))
print()

.  .  .  .  .  .  .  .  .  .  .  .  
.  $  $  $  $  $  $  .  .  .  .  .  
.  .  .  .  www.  $  $  $  .  .  .  
.  .  .  .  wwwwwwwww.  $  .  .  .  
.  .  .  .  wwwwwwwww.  $  $  .  .  
.  .  .  .  .  .  .  .  .  $  .  .  
.  .  .  .  .  .  .  .  www$  .  .  
.  wwwwwwwww.  .  .  .  www$  .  .  
.  wwwwwwwww.  .  .  .  www$  $  .  
.  .  .  .  .  .  .  .  www.  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  .  .  

